In [647]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [648]:
df = pd.read_csv('apps_policy.csv')

In [649]:
df.shape

(22977, 32)

In [650]:
## Create GLA buckets
def f(df):
    if df['gla'] < 75000:
        val = 'a. < 75k'
    elif df['gla'] < 250000:
        val = 'b. 75k - 250k'
    elif df['gla'] < 500000:
        val = 'c. 250k - 500k'
    elif df['gla'] < 1000000:
        val = 'd. 500k - 1M'
    elif df['gla'] >= 1000000:
        val = 'e. 1M+'
    else: val = 'error'
    return val

df['gla_buckets'] = df.apply(f, axis = 1)

## Create inflow buckets
def i(df):
    if df['avg_3_month_revenue'] < 10000:
        val = 'a. 0 - 10k'
    elif df['avg_3_month_revenue'] < 25000:
        val = 'b. 10k - 25k'
    elif df['avg_3_month_revenue'] < 50000:
        val = 'c. 25k - 50k'
    elif df['avg_3_month_revenue'] < 100000:
        val = 'd. 50k - 100k'
    elif df['avg_3_month_revenue'] < 500000:
        val = 'e. 100k - 500k'
    elif df['avg_3_month_revenue'] >= 500000:
        val = 'f. 500k+'
    else: val = 'none'
    return val

df['inflow_buckets'] = df.apply(i, axis = 1)

## Create sbfe buckets
def s(df):
    if df['sbfe_score'] < 800:
        val = 'a. <800'
    elif df['sbfe_score'] < 840:
        val = 'b. 800-840'
    elif df['sbfe_score'] < 860:
        val = 'c. 840-860'
    elif df['sbfe_score'] < 880:
        val = 'd. 860-880'
    elif df['sbfe_score'] >= 880:
        val = 'e. 880+'
    else: val = 'none'
    return val

df['sbfe_score_buckets'] = df.apply(s, axis = 1)

In [651]:
##Additional fields
df['Date'] = pd.to_datetime(df.fa_submitted_at)
df['approved_limit'] = df['gla'] *0.1
df['val'] = 1

## get only Sept submissions
submitted = df.loc[(df['fa_status'] != 'Pending')
                & (df['Date'] >= '2021-08-01') & (df['Date'] <= '2021-09-30') 
#                 ##& ((df['business_policy_type'] == 'STANDARD_POLICY') | (df['business_policy_type'].isnull()) 
# #                   ## & (df['fa_rejection_reason'] != 'Suspected fraud'))
                  ]

submitted.shape

(2522, 38)

In [652]:
## resolved (i.e., not in a pending state = 'docs required/submitted or kyc approved')
not_pending = ((submitted['fa_status'] != 'DOCUMENTS_SUBMITTED') & (submitted['fa_status'] != 'DOCUMENTS_REQUIRED') 
               & (submitted['fa_status'] != 'KYC Approved') & (submitted['fa_status'] != 'OPS_REVIEW')
               & (submitted['fa_status'] != 'Submitted'))
submitted = submitted.loc[not_pending]
submitted.shape

(2459, 38)

In [653]:
## rejected due to fraud
exclude_fraud = submitted['fa_rejection_reason'] != 'Suspected fraud'
submitted = submitted.loc[exclude_fraud]
submitted.shape

(2422, 38)

In [654]:
submitted['fa_status'].value_counts()

Rejected          1423
Admin Approved     958
Withdrawn           41
Name: fa_status, dtype: int64

In [655]:
## approved as SESA/YC/VIP
sesa_yc_vip = ((submitted['business_policy_type'] != 'SESA') & (submitted['business_policy_type'] != 'YC') & (submitted['business_policy_type'] != 'VIP'))
submitted = submitted.loc[sesa_yc_vip]
submitted.shape

(2231, 38)

In [687]:
submitted['fa_status'].value_counts()

Rejected          1423
Admin Approved     767
Withdrawn           41
Name: fa_status, dtype: int64

In [656]:
# approved not on exception
approved_not_on_exception = (((submitted['scorecard_approved'] == 1) | (submitted['bureau_approved'] == 1)) & (submitted['fa_status'] == 'Admin Approved'))
approved_noe_df = submitted.loc[approved_not_on_exception]
approved_noe_df.shape

(709, 38)

In [657]:
## approved on exception
approved_on_exception = (((submitted['scorecard_approved'] == 0) & (submitted['bureau_approved'] == 0)) & (submitted['fa_status'] == 'Admin Approved'))
approved_oe_df = submitted.loc[approved_on_exception]
approved_oe_df.shape

(58, 38)

In [718]:
submitted.loc[submitted['fa_status'] =='Admin Approved'][['gla_buckets', 'fa_id']].groupby('gla_buckets').count()

,fa_id
gla_buckets,
a. < 75k,27
b. 75k - 250k,131
c. 250k - 500k,157
d. 500k - 1M,152
e. 1M+,300


In [659]:
## Remaining rejected apps
remaining_rejected = ((submitted['fa_status'].isin(['Rejected', 'Withdrawn'])))
rejected_df = submitted.loc[remaining_rejected]
rejected_df.shape

(1464, 38)

In [660]:
## GLA >= $75k OR <= $250K
gla_75_250k = ((rejected_df['gla'] >= 75000) & (rejected_df['gla'] < 250000))
gla_df = rejected_df[gla_75_250k]
gla_df.shape

(228, 38)

In [661]:
## Remaining
remaining = gla_df

### Bureau + Comp

In [662]:
## have a confirmed bureau_record
confirmed_bureau = remaining['is_confirmed'].notnull()
confirmed_bureau_df = remaining[confirmed_bureau]
confirmed_bureau_df.shape

(154, 38)

In [663]:
## pass bureau policy but rejected on exception (incomplete application information, personal funds got them over $75k, very short runway, withdrawn, etc..)
pass_bureau_policy = confirmed_bureau_df['bureau_approved'] == 1
pass_bureau_df = confirmed_bureau_df.loc[pass_bureau_policy]
pass_bureau_df.shape
## uncomment to see who and why
#pass_bureau_df[['fa_id','fa_status', 'fa_rejection_reason']]

(9, 38)

In [664]:
## don't pass bureau_policy
no_pass_bureau_policy = confirmed_bureau_df['bureau_approved'] != 1
no_pass_bureau_df = confirmed_bureau_df.loc[no_pass_bureau_policy]
no_pass_bureau_df.shape

(145, 38)

In [688]:
## fail due to sbfe
fail_sbfe = (no_pass_bureau_df['sbfe_score'] < 880) | (no_pass_bureau_df['sbfe_score'].isnull())
sbfe_fail = no_pass_bureau_df.loc[fail_sbfe]
sbfe_fail.shape

(93, 38)

In [693]:
## remaining that fail dq
fail_dq = ((no_pass_bureau_df['sbfe_score'] >= 880) & (no_pass_bureau_df['sbfe_score'].notnull())) & ((no_pass_bureau_df['dpd_30_balance'] >= 1000) & (no_pass_bureau_df['dpd_30_balance'].notnull()))
fail_dq_df = no_pass_bureau_df.loc[fail_dq]
fail_dq_df.shape

(1, 38)

In [694]:
## remaining that fail bureau_limit
fail_limit = ((no_pass_bureau_df['sbfe_score'] >= 880) & (no_pass_bureau_df['sbfe_score'].notnull())) & ((no_pass_bureau_df['dpd_30_balance'] < 1000) | (no_pass_bureau_df['dpd_30_balance'].isnull()))
fail_limit_df = no_pass_bureau_df.loc[fail_limit]
fail_limit_df.shape

(51, 38)

In [695]:
## incremental approvals using competitor spend
have_comp_spend = no_pass_bureau_df['competitor_avg_3_month_spend'] >= 10000
incremental_approval_comp_df = fail_limit_df.loc[have_comp_spend]
incremental_approval_comp_df.shape
#see why they were rejected. almost all because of GLA!
#incremental_approval_comp_df[['fa_id','sbfe_score', 'dpd_30_balance', 'bureau_max_credit_limit', 'competitor_avg_3_month_spend', 'fa_rejection_reason']]

(9, 38)

In [705]:
no_comp_spend = no_pass_bureau_df['competitor_avg_3_month_spend'] < 10000
no_spend = fail_limit_df.loc[no_comp_spend]
no_spend[['fa_id','competitor_avg_3_month_spend']].mean()

fa_id                           20711.428571
competitor_avg_3_month_spend     4311.330595
dtype: float64

### Scorecard Approvals excluding incrementals from bureau + comp

In [669]:
scorecard_remaining = remaining.loc[~remaining['fa_id'].index.isin(incremental_approval_comp_df['fa_id'].index)]
scorecard_remaining.shape

(219, 38)

In [670]:
scorecard_remaining[['connection_type','fa_id']].groupby('connection_type').count()

,fa_id
connection_type,
Direct Connection,140
Manual Connection,79


In [671]:
## Scorecard approvals
scorecard_approved = scorecard_remaining['scorecard_approved'] == 1
scorecard_approved_df = scorecard_remaining[scorecard_approved]
scorecard_approved_df.shape

(177, 38)

In [672]:
scorecard_approved_df[['connection_type','fa_id']].groupby('connection_type').count()

,fa_id
connection_type,
Direct Connection,105
Manual Connection,72


In [707]:
directly_connected = scorecard_approved_df['connection_type'] == 'Direct Connection'
scorecard_approved_df_direct = scorecard_approved_df.loc[directly_connected]
scorecard_approved_df_direct.shape

(105, 38)

In [710]:
#has more than 100 trxns
trxns = (scorecard_approved_df_direct['all_trxns'] >= 100)
scorecard_approved_df_direct = scorecard_approved_df_direct.loc[trxns]
scorecard_approved_df_direct.shape

(63, 38)

In [711]:
#pivot risk score against inflows for directly connected apps
table = pd.pivot_table(scorecard_approved_df_direct, index='risk_score', columns='inflow_buckets', values='val', aggfunc=np.sum )
table

inflow_buckets,a. 0 - 10k,b. 10k - 25k,c. 25k - 50k,d. 50k - 100k,e. 100k - 500k,f. 500k+
risk_score,,,,,,
0.0,NaN,3.0,1.0,5.0,2.0,NaN
3.0,3.0,2.0,6.0,7.0,12.0,2.0
6.0,1.0,NaN,2.0,3.0,8.0,NaN
8.0,NaN,NaN,NaN,NaN,1.0,NaN
9.0,NaN,NaN,NaN,1.0,2.0,2.0


In [712]:
#What does this look like for approved businesses
all_approvals = submitted.loc[(submitted['fa_status'] == 'Admin Approved') 
                             & (submitted['connection_type'] == 'Direct Connection')
                             & (submitted['gla'] >= 75000) & (submitted['gla'] < 250000)]
table = pd.pivot_table(all_approvals, index='risk_score', columns='inflow_buckets', values='val', aggfunc=np.sum )
table

inflow_buckets,a. 0 - 10k,b. 10k - 25k,c. 25k - 50k,d. 50k - 100k,e. 100k - 500k,f. 500k+,none
risk_score,,,,,,,
0.0,NaN,NaN,4.0,5.0,5.0,NaN,22.0
3.0,1.0,1.0,NaN,5.0,4.0,4.0,3.0
5.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
6.0,NaN,NaN,1.0,NaN,5.0,1.0,3.0
8.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
13.0,NaN,NaN,NaN,3.0,2.0,1.0,1.0
16.0,NaN,1.0,1.0,NaN,2.0,2.0,5.0
19.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
29.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [713]:
##plots of risk segment and score
score_segment_plot = scorecard_approved_df_direct[['risk_segment', 'risk_score', 'fa_id']].groupby(['risk_segment', 'risk_score']).count()
risk_score_plot = scorecard_approved_df_direct[['risk_score', 'fa_id']].groupby(['risk_score']).count()
risk_segment_plot = scorecard_approved_df_direct[['risk_segment', 'fa_id']].groupby(['risk_segment']).count()
# risk_score_plot
#score_segment_plot
# score_segment_plot.plot.bar()
# risk_score_plot.plot.bar()
# risk_segment_plot.plot.bar()

#### Drawing a line in the sand

- Approve everything with either a risk score in (0, 3) OR a (risk_score in = 6 AND inflows >= 50k)
- AND that has been in business for >= 2 years

In [714]:
incremental_approvals_scorecard_inflows_df = scorecard_approved_df_direct.loc[(((scorecard_approved_df_direct['risk_score'].isin([0,3])) 
                                                                               | ((scorecard_approved_df_direct['risk_score'] == 6) 
                                                                                 & (scorecard_approved_df_direct['avg_3_month_revenue'] >= 50000)))
                                                                              & (scorecard_approved_df_direct['tib_years'] >= 2))]
incremental_approvals_scorecard_inflows_df[['fa_id', 'risk_score','avg_3_month_revenue', 'tib_years' ]]
incremental_approvals_scorecard_inflows_df.shape

(41, 38)

#### Results
- This is 69/105 or 65% incremental approval on the rejected apps
- We should apply this percentage to the manually connected 72 + the scorecard_approved percent, so we get 72*0.3*0.65 and we get 14 additional
- So a total of 83 on 177 scorecard approved apps
- Or a total of 83 on 219 remaining apps, 38%

### Lower bureau SBFE score excluding bureau + comp and scorecard_approvls

In [683]:
bureau_remaining = scorecard_remaining.loc[~scorecard_remaining['fa_id'].index.isin(incremental_approvals_scorecard_inflows_df['fa_id'].index)]
bureau_remaining.shape

(178, 38)

In [684]:
## have a confirmed bureau_record
confirmed_bureau_c = bureau_remaining['is_confirmed'].notnull()
confirmed_bureau_c_df = bureau_remaining[confirmed_bureau_c]
confirmed_bureau_c_df.shape

(118, 38)

In [720]:
pass_dq = ((confirmed_bureau_c_df['dpd_30_balance'] < 1000) | (confirmed_bureau_c_df['dpd_30_balance'].isnull()))
pass_limit = ((confirmed_bureau_c_df['competitor_avg_3_month_spend'] >= 10000) | (confirmed_bureau_c_df['bureau_max_credit_limit'] >= 10000))
pass_bureau = (confirmed_bureau_c_df['sbfe_score'] < 880)
pass_bureau_except_sbfe = confirmed_bureau_c_df.loc[(pass_dq) & (pass_limit) & (pass_bureau)]
pass_bureau_except_sbfe.shape

(22, 38)

In [721]:
pass_bureau_except_sbfe[['sbfe_score_buckets', 'fa_id']].groupby('sbfe_score_buckets').count()

,fa_id
sbfe_score_buckets,
b. 800-840,5
c. 840-860,9
d. 860-880,8
